In [ ]:
from tools import tools_map, query_user_for_details,gen_tools_desc
from prompt import gen_prompt, user_prompt
tools_dict= gen_tools_desc()
print(tools_dict)
print(user_prompt)

In [ ]:
from tools import tools_map, query_user_for_details
from prompt import gen_prompt, user_prompt
from model_provider import ModelProvider
from dotenv import load_dotenv
import dashscope
import os
import json

load_dotenv()

class Medical_Knowledgebase_Agent:
    def __init__(self):
        self.mp = ModelProvider()
        dashscope.api_key = "sk-f529539e3a50472fac783cf1e99fddef"
        os.environ["TAVILY_API_KEY"] = "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"

    def parse_thoughts(self, response, cur_request_time, max_request_time, debug):
        try:
            thoughts = response.get("thoughts")
            planning = thoughts.get("planning")
            reasoning = thoughts.get("reasoning")
            reflection = thoughts.get("reflection")
            history = thoughts.get("history")
            summary = thoughts.get("summary")
            observation = response.get("observation")
            prompt = f"planning: {planning}\nreasoning: {reasoning}\nreflection: {reflection}\nhistory: {history}\nobservation: {observation}\nsummary: {summary}"
            prompt += f"\n这是医疗知识库第{cur_request_time}次响应，最多执行{max_request_time}次\n"
            if debug:
                print(prompt)
            return prompt
        except Exception as e:
            print(f"parse_thoughts error: {e}")
            return ""

    def execute_action(self, action_name, action_args, debug):
        try:
            func = tools_map.get(action_name)
            result = func(**action_args)
            if debug:
                print(f"action_name: {action_name}, action_args: {action_args}")
            return result
        except Exception as e:
            print(f"调用工具异常： {e}")
            return str(e)

    def agent_execute(self, query, max_request_time=5, debug=False):
        cur_request_time = 0
        chat_history = []
        agent_scratch = ""

        while cur_request_time < max_request_time:
            cur_request_time += 1
            prompt = gen_prompt(query, agent_scratch)
            response = self.mp.chat(prompt, chat_history)

            if not response or not isinstance(response, dict):
                print(f"call llm exception, response is: {response}")
                continue

            action_info = response.get("action")
            action_name = action_info.get("name")
            action_args = action_info.get("parameters")
            if debug:
                print(f'-------------医学知识库Agent第{cur_request_time}次推断------------')
                print(response)
                print(json.dumps(response, ensure_ascii=False, indent=4))

            if action_name == "query_user_for_details":
                user_response = input(query_user_for_details(action_args["prompt"]))
                chat_history.append([action_args["prompt"], user_response])
                agent_scratch += f"query_user: {action_args['prompt']}user response: {user_response}"
                continue

            call_function_result = self.execute_action(action_name, action_args, debug)
            agent_scratch += f"observation: {response.get('observation')} execute action {action_name} result: {call_function_result}"
            assistant_msg = self.parse_thoughts(response, cur_request_time, max_request_time, debug)
            chat_history.append([user_prompt, assistant_msg])

            if action_name == "finish":
                final_answer = action_args.get("answer")
                break

        if cur_request_time == max_request_time:
            final_answer = "本次任务执行失败！未能提供相关医学知识"
        print(f"final_answer: {final_answer}")
        return final_answer

In [ ]:
# 使用类
agent = Medical_Knowledgebase_Agent()
max_request_time = 6
query = '你好，你能告诉我今天星期几吗？'
final_answer = agent.agent_execute(query, max_request_time=max_request_time,debug=True)
print(f"final_answer: {final_answer}")

In [10]:
# from tools import tools_map, query_user_for_details
# from prompt import gen_prompt, user_prompt
# from model_provider import ModelProvider
# from dotenv import load_dotenv
# import dashscope
# import os
# load_dotenv()
# mp = ModelProvider()

# dashscope.api_key = 'sk-f529539e3a50472fac783cf1e99fddef'
# os.environ["TAVILY_API_KEY"] = "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"
# # 解析模型返回的响应，提取关键信息并格式化为字符串
# def parse_thoughts(response):
#     try:
#         thoughts = response.get("thoughts")
#         observation = response.get("observation")
#         planning = thoughts.get("planning")
#         reasoning = thoughts.get("reasoning")
#         reflection = thoughts.get("reflection")
#         summary = thoughts.get("summary")
#         query_user = thoughts.get("query_user")
#         prompt = f"planning: {planning}reasoning: {reasoning}reflection: {reflection}observation: {observation}summary: {summary}query_user: {query_user}"
#         return prompt
#     except Exception as e:
#         print(f"parse_thoughts error: {e}")
#         return ""
#     return {}

# # 执行代理任务，与模型交互并处理结果
# def agent_execute(query, max_request_time):
#     cur_request_time = 0
#     chat_history = []
#     agent_scratch = ""
#     while cur_request_time < max_request_time:
#         cur_request_time += 1
#         prompt = gen_prompt(query, agent_scratch)
#         print('开始调用通义千问.....')
#         response = mp.chat(prompt, chat_history)
#         print(response)
#         if not response or not isinstance(response, dict):
#             print(f"call llm exception, response is: {response}")
#             continue
#         action_info = response.get("action")
#         action_name = action_info.get("name")
#         action_args = action_info.get("args")
#         print(f"action_name: {action_name}, action_args: {action_args}")
#         # thoughts = response.get("thoughts")
#         # planning = thoughts.get("planning")
#         # reasoning = thoughts.get("reasoning")
#         # reflection = thoughts.get("reflection")
#         # summary = thoughts.get("summary")
#         # observation = response.get("observation")
#         # print(f"observation: {observation}")
#         # print(f"planning: {planning}")
#         # print(f"reasoning: {reasoning}")
#         # print(f"reflection: {reflection}")
#         # print(f"summary: {summary}")
#         if action_name == "query_user_for_details":
#             user_response = input(query_user_for_details(action_args['prompt']))
#             chat_history.append([query_user_for_details(action_args['prompt']), user_response])
#             agent_scratch = agent_scratch + f"query_user: {query_user_for_details(action_args['prompt'])}user response: {user_response}"
#             continue
#         try:
#             func = tools_map.get(action_name)
#             call_function_result = func(**action_args)
#         except Exception as e:
#             print(f"调用工具异常： {e}")
#             call_function_result = f"{e}"
#         agent_scratch = agent_scratch + f"observation: {observation}execute action result: {call_function_result}"
#         assistant_msg = parse_thoughts(response)
#         chat_history.append([user_prompt, assistant_msg])
#         if action_name == "finish":
#             final_answer = action_args.get("answer")
#             print(f"final_answer: {final_answer}")
#             break
#         observation = response.get("observation")
#     if cur_request_time == max_request_time:
#         print("本次任务执行失败！")
#     else:
#         print("本次任务成功！")


In [4]:
from tools import tools_map, query_user_for_details
from prompt import gen_prompt, user_prompt
from model_provider import ModelProvider
from dotenv import load_dotenv
import dashscope
import os

load_dotenv()
mp = ModelProvider()

dashscope.api_key = "sk-f529539e3a50472fac783cf1e99fddef"
os.environ["TAVILY_API_KEY"] = "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"

def parse_thoughts(response, cur_request_time,max_request_time,debug):
    """解析模型返回的响应，提取关键信息并格式化为字符串，并根据debug参数决定是否打印"""
    try:
        thoughts = response.get("thoughts")
        planning = thoughts.get("planning")
        reasoning = thoughts.get("reasoning")
        reflection = thoughts.get("reflection")
        summary = thoughts.get("summary")
        observation = response.get("observation")
        prompt = f"planning: {planning}\nreasoning: {reasoning}\nreflection: {reflection}\nobservation: {observation}\nsummary: {summary}"
        prompt += f"\n这是医疗知识库第{cur_request_time}次响应，最多执行{max_request_time}次\n" 
        if debug:
            print(prompt)
        return prompt
    except Exception as e:
        print(f"parse_thoughts error: {e}")
        return ""

def execute_action(action_name, action_args, debug):
    """根据动作名称执行相应的工具函数，并根据debug参数决定是否打印动作信息"""
    try:
        func = tools_map.get(action_name)
        result = func(**action_args)
        if debug:
            print(f"action_name: {action_name}, action_args: {action_args}")
        return result
    except Exception as e:
        print(f"调用工具异常： {e}")
        return str(e)

def agent_execute(query, max_request_time, debug=False):
    """执行代理任务，与模型交互并处理结果，根据debug参数决定是否打印详细信息"""
    cur_request_time = 0
    chat_history = []
    agent_scratch = ""

    while cur_request_time < max_request_time:
        cur_request_time += 1
        prompt = gen_prompt(query, agent_scratch)
        response = mp.chat(prompt, chat_history)

        if not response or not isinstance(response, dict):
            print(f"call llm exception, response is: {response}")
            continue

        action_info = response.get("action")
        action_name = action_info.get("name")
        action_args = action_info.get("args")
        thoughts = response.get("thoughts")

        # if debug:
        #     print(f'-------------第{cur_request_time}次推断------------')
        #     print(f"observation: {response.get('observation')}")
        #     print(f"planning: {thoughts.get('planning')}")
        #     print(f"reasoning: {thoughts.get('reasoning')}")
        #     print(f"reflection: {thoughts.get('reflection')}")
        #     print(f"summary: {thoughts.get('summary')}")
        #     print(f"action_name: {action_name}, action_args: {action_args}")

        if action_name == "query_user_for_details":
            user_response = input(query_user_for_details(action_args["prompt"]))
            chat_history.append([action_args["prompt"], user_response])
            agent_scratch += f"query_user: {action_args['prompt']}user response: {user_response}"
            continue

        call_function_result = execute_action(action_name, action_args, debug)
        agent_scratch += f"observation: {response.get('observation')} execute action {action_name} result: {call_function_result}"
        assistant_msg = parse_thoughts(response, cur_request_time,max_request_time,debug)
        chat_history.append([user_prompt, assistant_msg])

        if action_name == "finish":
            final_answer = action_args.get("answer")
            final_answer = "".join([str(key) + str(value) for key, value in final_answer.items()])
            break
    if cur_request_time == max_request_time:
        final_answer="本次任务执行失败！,未能检索到相关知识"
    return "".join(final_answer)

# 示例执行，启用调试模式


In [6]:
import os, json
import dashscope
from prompt import user_prompt
from dashscope.api_entities.dashscope_response import Message
from transformers import AutoModelForCausalLM, AutoTokenizer,RobertaTokenizer, RobertaModel
import torch

dashscope.api_key = "sk-f529539e3a50472fac783cf1e99fddef"
os.environ["TAVILY_API_KEY"] = "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"


class ModelProvider(object):
    def __init__(self):
        # 设置最大重试次数
        self.device = "cuda"
        self.max_retry_time = 1
        self.local_run = True
        if self.local_run:
            # 从环境变量中获取API密钥和模型名称
            self.api_key = "sk-dd3b645e78c24ebbbfc34971555c05fa"
            self.model_name = "qwen-max"
            # 初始化dashscope客户端
            self._client = dashscope.Generation()
        else:
            # "Qwen/Qwen1.5-0.5B-chat" "Qwen/Qwen2-7B-Instruct"
            self.model_name = "Qwen/Qwen2-7B-Instruct"
            self.cache_dir = "/root/autodl-tmp/hug/"
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                trust_remote_code=True,
                cache_dir=self.cache_dir
            ).to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name, trust_remote_code=True, cache_dir=self.cache_dir)

        
    def chat(self, system_prompt, chat_history):
        cur_retry_time = 0
        while cur_retry_time < self.max_retry_time:
            cur_retry_time += 1
            if self.local_run:
                try:
                    # 构建消息列表，包括系统提示、用户提示和历史聊天记录
                    messages = [
                        Message(role="system", content=system_prompt),
                        Message(role="user", content=user_prompt)
                    ]
                    for his in chat_history:
                        messages.append(Message(role="user", content=his[0]))
                        messages.append(Message(role="assistant", content=his[1]))
                    # 调用模型API并获取响应
                    response = self._client.call(
                        model=self.model_name,
                        api_key=self.api_key,
                        messages=messages,
                        # result_format='message',  # set the result to be "message"  format.
                        # stream=True, # set streaming output
                        # incremental_output=True  # get streaming output incrementally
                        )
                    print(response)
                    # 解析模型响应并返回内容
                    content = self._parse_model_response(response)
                    return content
                except Exception as e:
                    print(f"call llm exception: {e}")
            else:
                try:
                    messages = [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                        ]
                    for his in chat_history:
                        messages.append({"role": "user", "content": his[0]})
                        messages.append({"role": "assistant", "content": his[1]})
                    text = self.tokenizer.apply_chat_template(
                        messages,
                        tokenize=False,
                        add_generation_prompt=True
                    )
                    model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)
                    generated_ids = self.model.generate(
                        model_inputs.input_ids,
                        max_new_tokens=512
                    )
                    generated_ids = [
                        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
                    ]

                    response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
                    print(response)
                    # 解析模型响应并返回内容
                    content = self._parse_model_response(response)
                    return content
                except Exception as e:
                    print(f"call llm exception: {e}")        
        return {}

    def _parse_model_response(self, response):
        """尝试解析模型响应为JSON格式"""
        text = response["output"]["text"] if self.local_run else response
        try:
            # 尝试直接解析文本为JSON
            return json.loads(text)
        except json.JSONDecodeError:
            # 如果直接解析失败，尝试查找被标记的JSON字符串
            json_start = text.find("```json")
            json_end = text.rfind("```")
            json_content = text[json_start + 7:json_end]
            return json.loads(json_content)



mp = ModelProvider()


In [17]:
prompt = """    你是一个智能知识库助手，专注于提供信息和建议。你可以自主决策，并在处理用户的询问时根据需要调用适当的工具。
    目标或其他条件:
    腱鞘炎该吃什么药
    限制条件说明:
    1.仅使用下面列出的动作，特别是在为用户提供医疗建议时，确保所有建议都是基于可用的医学数据和信息。
2.你可以根据用户的其他问题自主决策，处理非医疗内容的问题时只需基于已有知识和工具来回答。
    动作说明:你可以通过调用以下列出的工具来实现操作，你应尽可能快速、直接地利用已有知识或检索帮助用户解决问题：
    1.query_user_for_details:当大模型需要更多具体信息能提供个性化建议或解决方案时，用于向用户提问，以深入了解用户的需求、偏好或限制条件。当query_user不为空时使用该工具, args: [{"name": "prompt", "description": "向用户提出的问题，旨在澄清或细化其需求。", "type": "string"}]
2.finish:形成了完整的问答，用户表示满意, args: [{"name": "response", "description": "在此给出完整的问答信息", "type": "dict"}]
3.RAG_addition:该模块利用构建的知识图谱，能够提供疾病、症状、治疗方法等之间的详细关系和交互信息。它尤其适合用于理解复杂的医疗关系网络，比如药物副作用或并发症的详细信息。需结合RAG_addition模块提供基础数据。, args: [{"name": "query", "description": "用户查询的复杂医学条目或概念，涉及系统性知识理解和条件关联的查询。", "type": "string"}]
4.internet_search_function:通过联网查询获取最新的医学研究成果、临床试验报告及全球卫生政策更新，特别适用于需要最新医学信息的查询。此功能能弥补知识图谱和现有内部数据的不足。, args: [{"name": "query", "description": "需要了解最新科研成果或最新医学信息的查询，如新型疗法、新药上市信息或新兴病毒的防控措施。", "type": "string"}] 
工具使用逻辑：
1. 当用户的问题涉及医学相关知识或健康情况时，首先调用RAG_addition模块。可以提供广泛而深入的信息。
2. 如果以上模块提供的信息仍未完全满足用户需求，或者问题涉及最新医学研究成果和全球卫生政策更新，可以启动internet_search_function功能来查找最新医学数据或者别的领域的信息。
3. 在用户信息不完整时，使用query_user_for_details工具来询问用户详细情况以获取更多信息。
4. 当用户的问题不是关于医学的，请使用大模型自身的能力和internet_search_function联网搜索。

    资源说明:
    1.提供搜索和信息收集的互联网接入，以便获取最新的医疗或其他相关信息。
2.你是一个大语言模型，接受了大量医疗、科技、文化等领域的文本训练，可以帮助解答医疗及其他问题。
    策略说明:
    1.确保信息准确有效，直接为用户提供最有用的回答。
2.基于你检索到的知识或已有的内置知识提供回答，确保及时性和简洁性。
3.如果初次使用RAG检索的信息不足。如果涉及到别的领域或者具有时效性的信息，则进行联网搜索，最终如果仍没有匹配，返回'没有检索到合适的答案'。
    # agent_scratch:observation: 开始执行查询，以获取关于腱鞘炎治疗药物的详细信息。 execute action RAG_addition result: 腱鞘炎宜食的食物包括有：鸭蛋;鸭翅;鸡爪;鸡蛋
推荐食谱包括有：鸡肉蛋花汤;羊肉温补汤;草鱼豆腐;酱肉西兰花;鸡肉炒藕丝;竹筒蒸草鱼;羊肉煎包;鸡肉冬菜饼
腱鞘炎通常的使用的药品包括：双氯芬酸钠肠溶片；双氯芬酸二乙胺凝胶；布洛芬片；伤痛酊；布洛芬缓释胶囊；布洛芬缓释混悬液；Ⅰ；酮洛芬凝胶；双氯芬酸钠缓释片；阿西美辛缓释胶囊；依托芬那酯凝胶；复方七叶皂苷钠凝胶
    请以json格式进行响应，严格按照响应格式如下:
    
{
    "action": {
        "name": "动作名称",
        "args": {
            "args name": "执行动作所需参数的值"
        }
    },
    "thoughts": {
        "planning": "解决用户问题的具体实现步骤",
        "reflection": "建设性的自我批评与反思",
        "summary": "当前步骤的总结，用户端接受的是此字段的内容；如果是最后一步finish，在这里给出用户所需要的具体内容",
        "reasoning": "判断信息的合适性，是否需要继续搜索或着提供最终答案，如果信息合适，直接最终答案"
    },
    "observation": "观察当前任务的整体进度"
}

"""

In [ ]:
chat_history = ""
mp.chat(prompt, chat_history)

In [19]:
# from transformers import AutoModelForCausalLM, AutoTokenizer,RobertaTokenizer, RobertaModel
# from transformers import AutoModel
# import torch
# from peft import PeftModel
# from dashscope.api_entities.dashscope_response import Message
# Model_Name = "Qwen/Qwen1.5-0.5B-chat"
# device = "cuda"
# tokenizer = AutoTokenizer.from_pretrained(Model_Name, trust_remote_code=True,cache_dir="/root/autodl-tmp/hug/")
# model = AutoModelForCausalLM.from_pretrained(
#     Model_Name,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     cache_dir="/root/autodl-tmp/hug/"
# ).to(device)
# # "Qwen/Qwen2-0.5B"
# # "Qwen/Qwen1.5-0.5B-chat"

## RAG

In [1]:
import os
import faiss
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer, RobertaModel
from sentence_transformers import CrossEncoder


os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# 加载配置
with open('/root/autodl-tmp/MedicaAgent/agent/chatbotagent/config.json', encoding='utf-8') as config_file:
    config = json.load(config_file)


class RAGModule:
    def __init__(self):
        # embedding模型
        self.tokenizer = AutoTokenizer.from_pretrained(config['embedding_model_path'])
        self.model = AutoModel.from_pretrained(config['embedding_model_path'])
        self.index = faiss.read_index(config['vector_dbindex_path'])  # 加载FAISS索引文件
        self.top_k = config['rag_retrieval_top_k']                    # 加载top_k

        # 加载 roberta-base 模型
        self.roberta_tokenizer = RobertaTokenizer.from_pretrained(config['roberta-base_path'])
        self.roberta_model = RobertaModel.from_pretrained(config['roberta-base_path'])

        # 加载 cross-encoder 模型
        self.cross_encoder = CrossEncoder(config['cross-encoder_path'])

        # 加载文本块与索引的映射
        with open(config['text_mapping_path'], 'r', encoding='utf-8') as f:
            self.text_mapping = json.load(f)

    def encode_text(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # 使用 [CLS] token 的输出作为句子向量
        return outputs.last_hidden_state.mean(dim=1).numpy()

    def roberta_encode(self, texts):
        inputs = self.roberta_tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.roberta_model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

    def query(self, query_text):
        # 使用 FAISS 进行初步召回
        query_vector = self.encode_text(query_text)
        distances, indices = self.index.search(query_vector, self.top_k)  # 召回 top_k 个文本块

        # 获取初步召回的文本块
        candidate_texts = [self.text_mapping[str(idx)] for idx in indices[0]]

        # 使用 roberta-base 进行高级召回
        roberta_query_vector = self.roberta_encode([query_text])[0]
        roberta_text_vectors = self.roberta_encode(candidate_texts)
        similarities = np.dot(roberta_text_vectors, roberta_query_vector) / (
                    np.linalg.norm(roberta_text_vectors, axis=1) * np.linalg.norm(roberta_query_vector))

        # 获取与查询文本最相关的 5 个文本块
        top_indices = np.argsort(similarities)[-5:]
        refined_texts = [candidate_texts[i] for i in top_indices]

        # 使用 cross-encoder 进行重排序
        cross_encoder_scores = self.cross_encoder.predict([(query_text, text) for text in refined_texts])
        ranked_texts = [text for _, text in
                        sorted(zip(cross_encoder_scores, refined_texts), key=lambda x: x[0], reverse=True)]

        return ranked_texts



In [ ]:
rag_module = RAGModule()
test_query = "早上肚子疼"
rag_results_texts = rag_module.query(test_query)
print('context', rag_results_texts)

## KG 知识图谱

In [4]:
import py2neo
neo4j_user='neo4j'
neo4j_password='neo4jneo4j'
neo4j_graph = py2neo.Graph("bolt://localhost:7687", auth=(neo4j_user, neo4j_password))

In [ ]:
entity = "百日咳"
result_child = neo4j_graph.run("MATCH (n {name:'"+ entity+"'})-[r]->(m)  \
                RETURN id(r) As id, type(r) As type, \
                n.name As title1 ,m.name As title2,\
                id(n) As id1 ,id(m) As id2,\
                head(labels(n)) As type1, head(labels(m)) As type2").data()
result_parent = neo4j_graph.run("MATCH (n)-[r]->(m {name:'"+ entity+"'})  \
                                RETURN id(r) As id, type(r) As type, \
                                n.name As title1 ,m.name As title2,\
                                id(n) As id1 ,id(m) As id2,\
                                head(labels(n)) As type1, head(labels(m)) As type2").data()
print(result_child)

In [ ]:
from kg_module import KGmodule
kg_model = KGmodule()

In [ ]:
query = '腱鞘炎吃什么药？'
answer = kg_model.query(query)
print(answer)

In [10]:

# system_prompt='你是一名医疗知识专家'
# user_prompt = "肚子疼怎么办？"
# # messages = [
# #     Message(role="system", content=system_prompt),
# #     Message(role="user", content=user_prompt)
# # ]
# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(device)

# generated_ids = model.generate(
#     model_inputs.input_ids,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
# print(response)

In [ ]:

system_prompt='你是一名医疗知识专家'
user_prompt = "肚子疼怎么办？"
# messages = [
#     Message(role="system", content=system_prompt),
#     Message(role="user", content=user_prompt)
# ]
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

attention_mask = torch.ones(model_inputs.input_ids.shape,device=device)
# Generate the response
generated_ids = model.generate(
    model_inputs.input_ids,
    attention_mask=attention_mask,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id   # Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

In [ ]:
# 医疗助手的任务描述与限制
constraints = [
    "仅使用下面列出的动作，特别是在为用户提供医疗建议时，确保所有建议都是基于可用的医学数据和信息。",
    "你只能主动行动，这意味着在处理健康咨询时，你需要能够自主决定诊断、治疗建议和健康管理策略。",
    "你无法与物理对象或药物实体直接交互，因此在推荐药物或治疗时，你必须依赖于医学文献、临床指南和其他权威健康信息。",
    "你应通过询问用户来了解其详细的健康状况、症状、既往病史和其他相关医疗信息。"
]

resources = [
    "提供搜索和信息收集的互联网接入，以便获取最新的医疗信息。",
    "你是一个大语言模型，接受了大量医疗和健康文本的训练，这将有助于为用户提供详尽的医疗建议，减少对实时数据的依赖。"
]

strategies = [
    "不断地回顾和分析你的行为，确保你提供的医疗建议既安全又有效，符合客户的健康需求。",
    "在提供医疗建议时进行建设性的自我批评，考虑治疗的可行性、患者的特殊状况以及可能影响治疗效果的因素。",
    "反思你过去的决策和策略，如客户满意度反馈，不断完善你的医疗建议方案。",
    "每个动作执行都有代价，因此在提供医疗建议时，要平衡成本与收益，确保提供性价比高的选择。",
    "利用你的信息收集能力来寻找最新、最准确的医疗信息，以增强治疗方案的有效性。"
]

# 动作描述
actions_description = [
    "RAG模块：当用户询问的健康问题需要更深入的答案时，使用此模块。RAG模块结合检索和生成技术提供综合的答案。",
    "知识图谱增强模块：在需要详细理解疾病、症状、治疗方法及其相互关系时使用此模块，利用知识图谱提供精准的、上下文相关的信息。",
    "联网搜索功能：当内置的知识库和图谱不能完全满足信息需求时，使用联网搜索获取最新的医学研究和全球健康信息。"
]

# 动作选择逻辑
action_logic = """
1. 当用户的问题涉及需要详尽答案的复杂医学或健康情况时，首先考虑调用RAG模块。这个模块结合了检索和生成策略，可以提供广泛和深入的信息。
2. 如果用户的问题更加特定，例如需要了解某种病症的详细治疗方案或药物相互作用，且RAG模块提供的答案未能完全解决用户疑问，则启用知识图谱增强模块。此模块利用构建的医学知识图谱，提供精准和关联性强的信息。
3. 当以上两个模块提供的信息仍未能完全满足用户需求，或当问题涉及最新医学研究成果、全球卫生政策更新时，最后考虑使用联网搜索功能。这一步骤可以获取当前最新、最全面的医学数据和信息。
"""

# JSON格式响应模板
response_format_prompt = """
{
    "action": {
        "name": "动作名称",
        "args": {
            "args name": "执行动作所需参数的值"
        }
    },
    "thoughts":{
        "planning": "根据用户需求选择最合适的动作执行",
        "reflection": "反思已执行动作的效果，考虑是否需要进一步信息来优化建议",
        "summary": "对当前步骤的总结，以及为用户提供的医疗建议",
        "reasoning": "说明选择特定动作的逻辑，是否需要从用户处获取更多信息"
    },
    "observation": "观察整体进展和用户满意度"
}
"""

# 完整模板调用
def generate_prompt(query, agent_scratch):
    return prompt_template.format(
        query=query,
        constraints="\n".join(constraints),
        actions_description="\n".join(actions_description),
        resources="\n".join(resources),
        strategies="\n".join(strategies),
        agent_scratch=agent_scratch,
        response_format_prompt=response_format_prompt
    )


In [ ]:
from dashscope import Generation
from datetime import datetime
import random
import json


# 定义工具列表，模型在选择使用哪个工具时会参考工具的name和description
tools = [
    # 工具1 获取当前时刻的时间
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
            "parameters": {}  # 因为获取当前时间无需输入参数，因此parameters为空字典
        }
    },  
    # 工具2 获取指定城市的天气
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {  # 查询天气时需要提供位置，因此参数设置为location
                        "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。"
                    }
                }
            },
            "required": [
                "location"
            ]
        }
    }
]

# 模拟天气查询工具。返回结果示例：“北京今天是晴天。”
def get_current_weather(location):
    return f"{location}今天是晴天。 "

# 查询当前时间的工具。返回结果示例：“当前时间：2024-04-15 17:15:18。“
def get_current_time():
    # 获取当前日期和时间
    current_datetime = datetime.now()
    # 格式化当前日期和时间
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # 返回格式化后的当前时间
    return f"当前时间：{formatted_time}。"

# 封装模型响应函数
def get_response(messages):
    response = Generation.call(
        model='qwen-max',
        messages=messages,
        tools=tools,
        seed=random.randint(1, 10000),  # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
        result_format='message',  # 将输出设置为message形式
        api_key = "sk-dd3b645e78c24ebbbfc34971555c05fa",
        model_name = "qwen-max"
    )
    return response

def call_with_messages():
    print('\n')
    messages = [
            {
                "content": input('请输入：'),  # 提问示例："现在几点了？" "一个小时后几点" "北京天气如何？"
                "role": "user"
            }
    ]
    
    # 模型的第一轮调用
    first_response = get_response(messages)
    assistant_output = first_response.output.choices[0].message
    print(f"\n大模型第一轮输出信息：{first_response}\n")
    messages.append(assistant_output)
    if 'tool_calls' not in assistant_output:  # 如果模型判断无需调用工具，则将assistant的回复直接打印出来，无需进行模型的第二轮调用
        print(f"最终答案：{assistant_output.content}")
        return
    # 如果模型选择的工具是get_current_weather
    elif assistant_output.tool_calls[0]['function']['name'] == 'get_current_weather':
        tool_info = {"name": "get_current_weather", "role":"tool"}
        location = json.loads(assistant_output.tool_calls[0]['function']['arguments'])['properties']['location']
        tool_info['content'] = get_current_weather(location)
    # 如果模型选择的工具是get_current_time
    elif assistant_output.tool_calls[0]['function']['name'] == 'get_current_time':
        tool_info = {"name": "get_current_time", "role":"tool"}
        tool_info['content'] = get_current_time()
    print(f"工具输出信息：{tool_info['content']}\n")
    messages.append(tool_info)

    # 模型的第二轮调用，对工具的输出进行总结
    second_response = get_response(messages)
    print(f"大模型第二轮输出信息：{second_response}\n")
    print(f"最终答案：{second_response.output.choices[0].message['content']}")

if __name__ == '__main__':
    call_with_messages()
    